In [42]:
import os
import numpy as np
import pandas as pd
from math import ceil
from collections import deque
import math
import sys
global maxdays
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: C:\Users\Sebas\OneDrive\Documenten\School\3e jaar\Project CO


In [43]:
%cd \Users\\Sebas\\OneDrive\\Documenten\\School\\3e jaar\\Project CO

C:\Users\Sebas\OneDrive\Documenten\School\3e jaar\Project CO


In [44]:
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "\\Users\\Sebas\\OneDrive\\Documenten\\School\\3e jaar\\Project CO"
filename = 'CO_Case2401.txt'
filesol = 'CO_Case2401_sol'


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)

print(title)
print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)
print(IDLE_MACHINE_COSTS)




Machines:
[[  1   7 315]
 [  2   6 578]]

Locations:
[[  1 230 367]
 [  2 349 370]
 [  3 297 246]
 [  4 495 150]
 [  5 376 373]]

Requests:
[[ 1  3  1  3  1  1]
 [ 2  2  1  4  2  2]
 [ 3  2  1  4  1  2]
 [ 4  4  1  3  1  2]
 [ 5  3  1  2  2  1]
 [ 6  2  1  3  2  2]
 [ 7  3  1  2  1  1]
 [ 8  3  1  3  1  2]
 [ 9  4  1  2  1  1]
 [10  2  1  3  1  2]]

Technicians:
[[  1   3 930   5   1   1]
 [  2   5 898   7   1   1]]
Instance 1
5
15
750
10
250
100000
5
400
20000
{1: 315, 2: 578}


In [49]:
# from here we can try with the technicians!! code above this works for route with 10 trucks and everything installed zo vroeg mogelijk

def nearest_neighbor(point, unvisited_points, locations):
    """
    Find the nearest neighbor of a given point among a list of unvisited points.

    Args:
    - point: The point for which the nearest neighbor is to be found.
    - unvisited_points: List of unvisited points.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - nearest_point: The nearest neighbor of the given point.
    """
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_point in unvisited_points:
        candidate_point_id = int(candidate_point)
        if candidate_point_id != int(point):
            candidate_coordinates = get_location_coordinates(candidate_point_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = candidate_point_id
    return nearest_point

def get_location_coordinates(location_id, locations):
    """
    Get the coordinates of a location based on its ID.

    Args:
    - location_id: The ID of the location.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - coordinates: Tuple (x, y) representing the coordinates of the location.
    """
    for location in locations:
        if location[0] == location_id:
            return (location[1], location[2])
    return None



def calculate_distance(coordinates1, coordinates2):
    """
    Calculate the Euclidean distance between two points given their coordinates.

    Args:
    - coordinates1: Tuple (x1, y1) representing the coordinates of the first point.
    - coordinates2: Tuple (x2, y2) representing the coordinates of the second point.

    Returns:
    - distance: The Euclidean distance between the two points.
    """
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    distance = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
    print(f"Distance between {coordinates1} and {coordinates2}: {distance}")
    return distance



def get_location_details(requests, locations, request_id):
    """
    Get the location details (ID and coordinates) corresponding to the given request ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id: The request ID for which the location details are to be retrieved.

    Returns:
    - location_details: A tuple containing the location ID and its corresponding coordinates [(location_id, x_coordinate, y_coordinate)]
    - None if the request ID is not found.
    """
    for request in requests:
        if request[0] == request_id:  # Check if request ID matches
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  # Find location details by matching location ID
                    return (location_id, location[1], location[2])  # Return location details
    return None  # Return None if request ID not found

def calculate_request_distance(requests, locations, request_id1, request_id2):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - distance: The Euclidean distance between the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        print(f"Location corresponding to Request ID {request_id1}: ID={location1[0]}, Coordinates=({location1[1]}, {location1[2]})")
        print(f"Location corresponding to Request ID {request_id2}: ID={location2[0]}, Coordinates=({location2[1]}, {location2[2]})")
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

# Example usage:
request_id1_input = 3
request_id2_input = 5

distance = calculate_request_distance(requests, locations, request_id1_input, request_id2_input)
if distance is not None:
    print(f"Distance between Request ID {request_id1_input} and Request ID {request_id2_input}: {distance:.2f}")
else:
    print("One or both request IDs not found.")
    
    
    
def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids

unique_location_ids = set(req[1] for req in requests)

# Iterate over unique location IDs
for location_id in unique_location_ids:
    # Get request IDs for the current location ID
    request_ids_for_location = get_requests_for_location(requests, location_id)
    # Print location ID and corresponding request IDs
    print(f"Requests belonging to Location ID {location_id}: {request_ids_for_location}")
    
    
def group_requests_by_location(requests):
    """
    Group requests by their location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]

    Returns:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    location_groups = {}
    for request in requests:
        request_id, location_id = request[0], request[1]
        if location_id not in location_groups:
            location_groups[location_id] = []
        location_groups[location_id].append(request_id)
    return location_groups

def print_location_groups(location_groups):
    """
    Print the location groups.

    Args:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    for location_id, request_ids in location_groups.items():
        print(f"Location ID {location_id}: {request_ids}")

location_groups = group_requests_by_location(requests)
print_location_groups(location_groups)



def calculate_request_weight(request_id, requests, machines):
    request = [req for req in requests if req[0] == request_id][0]
    machine_kind_id = request[4]
    num_requested_machines = request[5]
    machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
    return machine_size * num_requested_machines



def check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines):
    depot_loc_ID = depot[0]
    truck_loads = []
    current_truck_load = []
    current_truck_weight = 0
    
    for request_id, location_id, _, _, machine_kind_id, num_requested_machines in requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if current_truck_weight + request_weight > TRUCK_CAPACITY:
            if current_truck_load:
                truck_loads.append(current_truck_load)
            current_truck_load = [(request_id, request_weight)]
            current_truck_weight = request_weight
        else:
            current_truck_load.append((request_id, request_weight))
            current_truck_weight += request_weight
    
# Example usage
check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines)


def get_truck_routes_with_days(locations, depot, truck_capacity, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    current_route_distance = 0
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        request_distance = calculate_request_distance(request_id, location_id) #hier moeten de juiste location coordinaten nog bij
        
        if not current_truck_route:
            current_truck_route.append([request_id])
            current_truck_weight = request_weight
            current_route_distance = request_distance
        elif current_truck_weight + request_weight <= truck_capacity:
            # Check if adding the current request exceeds the truck capacity
            # If not, add it to the current truck route
            combined_request_ids = current_truck_route[-1] + [request_id]
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= truck_capacity:
                current_truck_route[-1] = combined_request_ids
                current_truck_weight = combined_weight
            else:
                truck_routes.append(current_truck_route)
                current_truck_route = [[request_id]]
                current_truck_weight = request_weight
        else:
            # If adding the current request exceeds the truck capacity,
            # start a new truck route with the current request
            truck_routes.append(current_truck_route)
            current_truck_route = [[request_id]]
            current_truck_weight = request_weight
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append(current_truck_route)
    
    return truck_routes


def calculate_delivery_days(truck_routes, requests):
    delivery_days = {}
    for truck_route in truck_routes:
        for location_id, request_ids in enumerate(truck_route, start=1):
            for request_id in request_ids:
                delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
                delivery_days[request_id] = delivery_day  # Record the delivery day for each request
    return delivery_days



# Calculate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate delivery days using truck routes
delivery_days = calculate_delivery_days(truck_routes, requests)




## weet niet of deze nog klopt na aanpassing delivery/installation days
def calculate_idle_days(requests, delivery_days):
    """
    Calculate the idle days for each machine based on delivery and installation days.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, delivery_start_day, delivery_end_day, machine_kind_id, num_requested_machines]
    - installations: List of installations where each installation is represented as a list [installation_day, machine_kind_id]

    Returns:
    - idle_days: Dictionary where keys are machine kind IDs and values are lists of idle days.
    """
    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
        machine_kind_id = requests[request_id - 1][4]  # Machine kind ID is at index 4
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

# Calculate idle days
idle_days = calculate_idle_days(requests, delivery_days)


def assign_technicians(delivery_dates, technicians, requests, max_consecutive_days):
    technician_assignments = {}
    consecutive_days = {}
    
    for request_id, installation_day in delivery_dates.items():
        delivery_day = requests[request_id - 1][2]
        earliest_assignment_day = delivery_day + 1
        
        for technician in technicians:
            location_id = technician[1]
            if technician[1] == location_id:
                skill_set = technician[4:]
                if skill_set[requests[request_id - 1][4] - 1] == 1:
                    earliest_available_day = max(earliest_assignment_day, installation_day)
                    
                    if technician[0] not in technician_assignments:
                        technician_assignments[technician[0]] = []
                        consecutive_days[technician[0]] = 0
                    
                    if consecutive_days[technician[0]] < max_consecutive_days:
                        if technician[0] not in consecutive_days:
                            consecutive_days[technician[0]] = 1
                        else:
                            consecutive_days[technician[0]] += 1
                            
                        technician_assignments[technician[0]].append((request_id, earliest_available_day))
                        break
                        
    return technician_assignments

# Example usage with the maximum consecutive days constraint
max_consecutive_days = 5

technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days)



# Print technician assignments
for technician_id, assigned_requests in technician_assignments.items():
    technician = [tech for tech in technicians if tech[0] == technician_id][0]
    skill_set = technician[4:]  # Get the skill set of the technician (from index 4 onwards)
    # Get the indices of the skill set where the value is 1
    skill_set_indices = [index + 1 for index, value in enumerate(skill_set) if value == 1]
    print(f"Technician {technician_id} skill set: {skill_set_indices} | Assigned Requests:")
    for request_id, assignment_day in assigned_requests:
        print(f"  - Request {request_id} (Earliest Assignment Day: {assignment_day})")





# Function to calculate penalties for idle days
def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1)  # Subtract 1 for the day of delivery
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties


# Function to implement constraints regarding maximum consecutive working days and total distance per day
def apply_constraints(technician_assignments):
    # Implement constraints here
    pass



def print_truck_routes(truck_routes, requests, machines, idle_days, delivery_days):
    for idx, truck_route in enumerate(truck_routes, start=1):
        print(f"Truck {idx} route:")
        total_weight = 0
        total_distance = 0
        for location_id, request_ids in enumerate(truck_route, start=1):
            print(f"  Location ID {location_id}: ", end="")
            for i, request_id in enumerate(request_ids):
                if i > 0:
                    print(", ", end="")
                machine_kind_id = requests[request_id - 1][4]
                print(f"Request {request_id} (Idle days: {idle_days[machine_kind_id]}, Delivery Day: {delivery_days[request_id]})", end="")
                request_weight = calculate_request_weight(request_id, requests, machines)
                total_weight += request_weight
                request_distance = calculate_distance(location_id,)
            print()
        print(f"  Total weight: {total_weight}")
        print(f"Total distance: {total_distance}")


technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days)
penalties = calculate_penalties(delivery_days, requests)
apply_constraints(technician_assignments)



# Example usage
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)
print_truck_routes(truck_routes, requests, machines, idle_days, delivery_days)


##Probeersel, werkt niet
def calculate_truck_route_distances(truck_routes, locations):
    """
    Calculate the total distance traveled by each truck route.

    Args:
    - truck_routes: List of truck routes, where each route is a list of location IDs.
    - locations: A dictionary where the keys are location IDs and the values are tuples of coordinates (x, y).

    Returns:
    - route_distances: A list containing the total distance traveled by each truck route.
    """
    route_distances = []
    for route in truck_routes:
        total_distance = calculate_route_distance(route, locations)
        route_distances.append(total_distance)
    return route_distances

    ##Probeersel, werkt niet
def calculate_route_distance(route, locations):
    """
    Calculate the total distance traveled by a given truck route.

    Args:
    - route: The route of the truck as a list of location IDs.
    - locations: A dictionary where the keys are location IDs and the values are tuples of coordinates (x, y).

    Returns:
    - total_distance: The total distance traveled by the truck route.
    """
    total_distance = 0
    for i in range(len(route) - 1):
        location1 = locations[route[i]]
        location2 = locations[route[i + 1]]
        distance = calculate_request_distance(location1, location2)
        total_distance += distance
    return total_distance

# Example usage:
route_distances = calculate_truck_route_distances(truck_routes, locations)
print("Total distance traveled by each truck route:", route_distances)

# Example usage:
route_distances = calculate_truck_route_distances(truck_routes, locations)
print("Total distance traveled by each truck route:", route_distances)


# Example usage:
total_distance_truck_routes = calculate_truck_routes_distance(truck_routes, locations_dict)
print("Total distance traveled by all truck routes:", total_distance_truck_routes)



# kijk goed naar requests, daarna technicians. voor nu alles zo vroeg mogelijk laten bezorgen


Location corresponding to Request ID 3: ID=2, Coordinates=(349, 370)
Location corresponding to Request ID 5: ID=3, Coordinates=(297, 246)
Distance between (349, 370) and (297, 246): 134.46189051177288
Distance between Request ID 3 and Request ID 5: 134.46
Requests belonging to Location ID 2: [2, 3, 6, 10]
Requests belonging to Location ID 3: [1, 5, 7, 8]
Requests belonging to Location ID 4: [4, 9]
Location ID 3: [1, 5, 7, 8]
Location ID 2: [2, 3, 6, 10]
Location ID 4: [4, 9]
Technician 1 skill set: [1, 2] | Assigned Requests:
  - Request 1 (Earliest Assignment Day: 2)
  - Request 2 (Earliest Assignment Day: 2)
  - Request 3 (Earliest Assignment Day: 2)
  - Request 4 (Earliest Assignment Day: 2)
  - Request 5 (Earliest Assignment Day: 2)
Technician 2 skill set: [1, 2] | Assigned Requests:
  - Request 6 (Earliest Assignment Day: 2)
  - Request 7 (Earliest Assignment Day: 2)
  - Request 8 (Earliest Assignment Day: 2)
  - Request 9 (Earliest Assignment Day: 2)
  - Request 10 (Earliest Assi

TypeError: calculate_distance() missing 1 required positional argument: 'coordinates2'

In [46]:
# Print the schedule for trucks and technicians; zonder toevoegingen
def print_schedule(truck_routes, technician_assignments, delivery_days, requests):
    current_day = 1
    while True:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route in truck_routes if delivery_days[route[0][0]] == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
            for truck_index, route in enumerate(truck_routes_for_day, start=1):
                request_ids = " ".join(str(request_id) for request_id in route)
                print(f"{truck_index} {request_ids}")
        else:
            print("NUMBER_OF_TRUCKS = 0")
        
        # Print technician assignments for the current day
        technician_assignments_for_day = [(tech_id, request_id) for tech_id, assignments in technician_assignments.items() for request_id, day in assignments if day == current_day]
        num_technicians_for_day = len(set(tech_id for tech_id, request_id in technician_assignments_for_day))
        if num_technicians_for_day > 0:
            print(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}")
            for tech_id, request_id in technician_assignments_for_day:
                print(f"{tech_id} {request_id}")
        else:
            print("NUMBER_OF_TECHNICIANS = 0")
        
        # Check if there are no more routes or assignments for the next day
        if not truck_routes_for_day and not technician_assignments_for_day:
            break
        
        current_day += 1

# Example usage
print_schedule(truck_routes, technician_assignments, delivery_days, requests)






DAY = 1
NUMBER_OF_TRUCKS = 10
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
6 [6]
7 [7]
8 [8]
9 [9]
10 [10]
NUMBER_OF_TECHNICIANS = 0

DAY = 2
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 2
1 1
1 2
1 3
1 4
1 5
2 6
2 7
2 8
2 9
2 10

DAY = 3
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 0


In [47]:
print("DATASET = VeRoLog solver challenge 2019")
print(f"NAME = {filename}")  # Gebruik de bestandsnaam van de instantie als de naam van de oplossing

def calculate_total_truck_distance(truck_routes, locations):
    """
    Bereken de totale afstand afgelegd door alle vrachtwagenroutes.

    Args:
    - truck_routes: Een lijst van vrachtwagenroutes, waar elke route een lijst van locatie-IDs is.
    - locations: Een dictionary waarbij de sleutels locatie-IDs zijn en de waarden tuples van coördinaten (x, y).

    Returns:
    - total_distance: De totale afstand afgelegd door alle vrachtwagenroutes.
    """
    total_distance = 0
    for route in truck_routes:
        total_distance += calculate_route_distance(route, locations)
    return total_distance

def calculate_route_distance(route, locations):
    """
    Bereken de totale afstand afgelegd door een vrachtwagenroute.

    Args:
    - route: De route van de vrachtwagen als een lijst van locatie-IDs.
    - locations: Een dictionary waarbij de sleutels locatie-IDs zijn en de waarden tuples van coördinaten (x, y).

    Returns:
    - total_distance: De totale afstand afgelegd door de vrachtwagenroute.
    """
    total_distance = 0
    for i in range(len(route) - 1):
        location1 = locations[route[i]]
        location2 = locations[route[i + 1]]
         
        total_distance += calculate_distance(location1, location2)
    return total_distance


# Bereken en print de kenmerken van de oplossing
locations_dict = {location[0]: (location[1], location[2]) for location in locations}
TRUCK_DISTANCE = calculate_total_truck_distance(truck_routes, locations_dict)
##NUMBER_OF_TRUCK_DAYS = bereken_aantal_vrachtwagendagen(truck_routes)  # Vervang door de werkelijke berekening
##NUMBER_OF_TRUCKS_USED = bereken_aantal_gebruikte_vrachtwagens(truck_routes)  # Vervang door de werkelijke berekening
##TECHNICIAN_DISTANCE = bereken_totale_technicusafstand(technician_assignments, locations)  # Vervang door de werkelijke berekening
##NUMBER_OF_TECHNICIAN_DAYS = bereken_aantal_technicusdagen(technician_assignments)  # Vervang door de werkelijke berekening
##NUMBER_OF_TECHNICIANS_USED = bereken_aantal_gebruikte_technici(technician_assignments)  # Vervang door de werkelijke berekening
##IDLE_MACHINE_COSTS = bereken_totale_idle_kosten(idle_days, machines)  # Vervang door de werkelijke berekening
##TOTAL_COST = bereken_totale_kosten(TRUCK_DISTANCE, NUMBER_OF_TRUCK_DAYS, NUMBER_OF_TRUCKS_USED, TECHNICIAN_DISTANCE, NUMBER_OF_TECHNICIAN_DAYS, NUMBER_OF_TECHNICIANS_USED, IDLE_MACHINE_COSTS)  # Vervang door de werkelijke berekening

# Print de oplossingen
print(f"TRUCK_DISTANCE = {TRUCK_DISTANCE}")
print(f"Route: {route}, Distance: {calculate_route_distance(route, locations)}")
print(f"Location 1: {location1}, Location 2: {location2}")
##print(f"NUMBER_OF_TRUCK_DAYS = {NUMBER_OF_TRUCK_DAYS}")
##print(f"NUMBER_OF_TRUCKS_USED = {NUMBER_OF_TRUCKS_USED}")
##print(f"TECHNICIAN_DISTANCE = {TECHNICIAN_DISTANCE}")
##print(f"NUMBER_OF_TECHNICIAN_DAYS = {NUMBER_OF_TECHNICIAN_DAYS}")
##print(f"NUMBER_OF_TECHNICIANS_USED = {NUMBER_OF_TECHNICIANS_USED}")
##print(f"IDLE_MACHINE_COSTS = {IDLE_MACHINE_COSTS}")
##print(f"TOTAL_COST = {TOTAL_COST}")

# Print de geplande routes voor elke dag
current_day = 1
while True:
    print()  # Print een lege regel voor elke dag
    print(f"DAY = {current_day}")
    
    # Print de vrachtroutes voor de huidige dag
    truck_routes_for_day = [route for route in truck_routes if delivery_days[route[0][0]] == current_day]
    num_trucks_for_day = len(truck_routes_for_day)
    if num_trucks_for_day > 0:
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            request_ids = " ".join(str(request_id) for request_id in route)
            print(f"{truck_index} {request_ids}")
    else:
        print("NUMBER_OF_TRUCKS = 0")
    
    # Print de toegewezen technici voor de huidige dag
    technician_assignments_for_day = [(tech_id, request_id) for tech_id, assignments in technician_assignments.items() for request_id, day in assignments if day == current_day]
    num_technicians_for_day = len(set(tech_id for tech_id, request_id in technician_assignments_for_day))
    if num_technicians_for_day > 0:
        print(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}")
        for tech_id, request_id in technician_assignments_for_day:
            print(f"{tech_id} {request_id}")
    else:
        print("NUMBER_OF_TECHNICIANS = 0")
    
    # Controleer of er geen routes of toewijzingen meer zijn voor de volgende dag
    if not truck_routes_for_day and not technician_assignments_for_day:
        break
    
    current_day += 1

DATASET = VeRoLog solver challenge 2019
NAME = CO_Case2401.txt
TRUCK_DISTANCE = 0
Route: [[10]], Distance: 0


NameError: name 'location1' is not defined

In [48]:
print("DATASET = VeRoLog solver challenge 2019")
print(f"NAME = {filename}")  # Use the filename of the instance as the name of the solution

# Construct truck_routes
truck_routes = [
    [[1]],  # Truck 1 route: Request 1
    [[2]],  # Truck 2 route: Request 2
    [[3]],  # Truck 3 route: Request 3
    [[4]],  # Truck 4 route: Request 4
    [[5]],  # Truck 5 route: Request 5
    [[6]],  # Truck 6 route: Request 6
    [[7]],  # Truck 7 route: Request 7
    [[8]],  # Truck 8 route: Request 8
    [[9]],  # Truck 9 route: Request 9
    [[10]], # Truck 10 route: Request 10
]

def calculate_total_truck_distance(truck_routes, locations):
    """
    Calculate the total distance traveled by all truck routes.

    Args:
    - truck_routes: A list of truck routes, where each route is a list of location IDs.
    - locations: A dictionary where the keys are location IDs and the values are tuples of coordinates (x, y).

    Returns:
    - total_distance: The total distance traveled by all truck routes.
    """
    total_distance = 0
    for route in truck_routes:
        print(f"Route: {route}")
        print(f"locations: {locations}")
        route_distance = calculate_route_distance(route, locations)
        print(f"Route distance: {route_distance}")
        total_distance += route_distance
    return total_distance

# Calculate and print the characteristics of the solution
locations_dict = {location[0]: (location[1], location[2]) for location in locations}
TRUCK_DISTANCE = calculate_total_truck_distance(truck_routes, locations_dict)

def calculate_route_distance(route, locations):
    """
    Bereken de totale afstand afgelegd door een vrachtwagenroute.

    Args:
    - route: De route van de vrachtwagen als een lijst van locatie-IDs.
    - locations: Een dictionary waarbij de sleutels locatie-IDs zijn en de waarden tuples van coördinaten (x, y).

    Returns:
    - total_distance: De totale afstand afgelegd door de vrachtwagenroute.
    """
    total_distance = 0
    for i in range(len(route) - 1):
        location1 = locations[route[i]]
        location2 = locations[route[i + 1]]
        total_distance += calculate_distance(location1, location2)  # Gebruik de bestaande calculate_distance-functie
        print(f"Location 1: {locations[route[i]]}, Location 2: {locations[route[i + 1]]}")  # Print de locaties direct vanuit de dictionary
    return total_distance

# Print the characteristics of the solution
print(f"TRUCK_DISTANCE = {TRUCK_DISTANCE}")

# Print the planned routes for each day
current_day = 1
while True:
    print()  # Print an empty line for each day
    print(f"DAY = {current_day}")
    
    # Print the truck routes for the current day
    truck_routes_for_day = [route for route in truck_routes if delivery_days[route[0][0]] == current_day]
    num_trucks_for_day = len(truck_routes_for_day)
    if num_trucks_for_day > 0:
        print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
        for truck_index, route in enumerate(truck_routes_for_day, start=1):
            request_ids = " ".join(str(request_id) for request_id in route)
            print(f"{truck_index} {request_ids}")
    else:
        print("NUMBER_OF_TRUCKS = 0")
    
    # Print the assigned technicians for the current day
    technician_assignments_for_day = [(tech_id, request_id) for tech_id, assignments in technician_assignments.items() for request_id, day in assignments if day == current_day]
    num_technicians_for_day = len(set(tech_id for tech_id, request_id in technician_assignments_for_day))
    if num_technicians_for_day > 0:
        print(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}")
        for tech_id, request_id in technician_assignments_for_day:
            print(f"{tech_id} {request_id}")
    else:
        print("NUMBER_OF_TECHNICIANS = 0")
    
    # Check if there are no more routes or assignments for the next day
    if not truck_routes_for_day and not technician_assignments_for_day:
        break
    
    current_day += 1


DATASET = VeRoLog solver challenge 2019
NAME = CO_Case2401.txt
Route: [[1]]
locations: {1: (230, 367), 2: (349, 370), 3: (297, 246), 4: (495, 150), 5: (376, 373)}
Route distance: 0
Route: [[2]]
locations: {1: (230, 367), 2: (349, 370), 3: (297, 246), 4: (495, 150), 5: (376, 373)}
Route distance: 0
Route: [[3]]
locations: {1: (230, 367), 2: (349, 370), 3: (297, 246), 4: (495, 150), 5: (376, 373)}
Route distance: 0
Route: [[4]]
locations: {1: (230, 367), 2: (349, 370), 3: (297, 246), 4: (495, 150), 5: (376, 373)}
Route distance: 0
Route: [[5]]
locations: {1: (230, 367), 2: (349, 370), 3: (297, 246), 4: (495, 150), 5: (376, 373)}
Route distance: 0
Route: [[6]]
locations: {1: (230, 367), 2: (349, 370), 3: (297, 246), 4: (495, 150), 5: (376, 373)}
Route distance: 0
Route: [[7]]
locations: {1: (230, 367), 2: (349, 370), 3: (297, 246), 4: (495, 150), 5: (376, 373)}
Route distance: 0
Route: [[8]]
locations: {1: (230, 367), 2: (349, 370), 3: (297, 246), 4: (495, 150), 5: (376, 373)}
Route dista

In [9]:
# alles hieronder kan weg

In [10]:
def get_request_ID(nearest_loc_ID, requests):
    for req in requests:
        if req[1] == nearest_loc_ID:
            return req[0]  # Return the request ID associated with the nearest location ID
    return None  # Return None if the nearest location ID is not found in requests

In [11]:
def solve_vrp_by_location(locations, depot, truck_capacity, truck_max_distance, truck_distance_cost, truck_day_cost, truck_cost, technician_distance_cost, technician_day_cost, technician_cost, requests, weights):
    depot_loc_ID = depot[0]
    routes = {'trucks': [], 'technicians': []}
    delivery_dates = {}
    idle_machine_costs = {}
    day = 0
    
    location_groups = group_requests_by_location(requests)

    while location_groups and day < DAYS:
        day += 1
        truck_routes = []
        
        truck_used = False
        truck_distance_traveled = 0
        truck_daily_distance_traveled = 0
        truck_daily_cost = 0
        overall_truck_cost = 0
        
        for location_id, request_ids in location_groups.items():
            location_requests = [(request_id, weight) for request_id, weight in requests if request_id in request_ids]
            
            # Check if the truck has enough capacity to deliver all requests to this location
            total_weight = sum(weight for _, weight in location_requests)
            if total_weight <= truck_capacity:
                if not truck_used:
                    truck_used = True
                    truck_daily_cost = truck_day_cost
                    overall_truck_cost += truck_cost
                truck_distance_traveled += calculate_distance(locations[depot_loc_ID][1:], locations[location_id][1:])
                truck_daily_distance_traveled += calculate_distance(locations[depot_loc_ID][1:], locations[location_id][1:])
                truck_daily_cost += truck_distance_cost * truck_distance_traveled
                truck_routes.append([request_id for request_id, _ in location_requests])
                del location_groups[location_id]
            else:
                print(f"Truck capacity exceeded for requests at Location ID {location_id}")
                # Handle the case where truck capacity is exceeded for this location
        
        routes['trucks'].append(truck_routes)
        
    total_truck_distance = sum(truck_distance_traveled for truck_routes in routes['trucks'] for truck_route in truck_routes for location_id in truck_route)
    total_cost = calculate_total_cost(overall_truck_cost, 0, idle_machine_costs, routes)
    
    print_output(depot_loc_ID, routes, total_truck_distance, 0, total_cost, overall_truck_cost, 0, idle_machine_costs)
    
    return routes, total_truck_distance, 0, total_cost


In [ ]:
def solve_vrp(locations, depot, truck_capacity, truck_max_distance, truck_distance_cost, truck_day_cost, truck_cost, technician_distance_cost, technician_day_cost, technician_cost, requests, weights):
    depot_loc_ID = depot[0]  
    unvisited_request_ids = [req[0] for req in requests]  
    current_location = depot_loc_ID  
    routes = {'trucks': [], 'technicians': []}
    
    delivery_dates = {}  
    idle_machine_costs = {}  
    
    day = 0
    
    while unvisited_request_ids and day < DAYS:  # Iterate over multiple days
        day += 1
        truck_routes = []
        technician_routes = []
        
        # Reset variables for the new day
        truck_used = False
        truck_distance_traveled = 0
        truck_daily_distance_traveled = 0
        truck_daily_cost = 0
        overall_truck_cost = 0
        
        technician_home_location = [9999, 9999]  
        
        technician_used = False
        technician_daily_distance_traveled = 0
        technician_daily_cost = 0
        overall_technician_cost = 0

        while unvisited_request_ids and (not truck_used or truck_daily_distance_traveled <= truck_max_distance):
            nearest = nearest_neighbor(current_location, unvisited_request_ids)
            nearest_request = [req for req in requests if req[0] == nearest][0]
            distance_to_nearest = distance_between_locations(current_location, nearest_request[1], locations)
            
            # Check if adding the next request exceeds the truck's maximum distance or capacity
            if (not truck_used) or (truck_used and (truck_daily_distance_traveled + distance_to_nearest > truck_max_distance)) or (sum(TRUCK_COST for req in truck_routes[-1]) + TRUCK_COST > truck_capacity):
                if truck_used:
                    truck_routes.append([0])  # Return to depot
                truck_used = True
                truck_daily_distance_traveled = 0
                truck_daily_cost = truck_day_cost
                overall_truck_cost += truck_cost
            
            truck_distance_traveled += distance_to_nearest
            truck_daily_distance_traveled += distance_to_nearest
            current_location = nearest_request[1]
            unvisited_request_ids.remove(nearest)
            
            truck_daily_cost += truck_distance_cost * distance_to_nearest
            
            if truck_daily_distance_traveled > truck_max_distance:
                truck_daily_cost += truck_day_cost
                truck_daily_distance_traveled = 0
        
            if truck_routes:
                truck_routes[-1].append(nearest)
            else:
                truck_routes.append([nearest])
        
        while unvisited_request_ids and technician_used:
            nearest = nearest_neighbor(current_location, unvisited_request_ids)
            nearest_request = [req for req in requests if req[0] == nearest][0]
            distance_to_nearest = distance_between_locations(current_location, nearest_request[1], locations)
            
            technician_daily_distance_traveled += distance_to_nearest
            technician_daily_cost += technician_distance_cost * distance_to_nearest
            
            technician_routes[-1].append(nearest)
            current_location = nearest_request[1]
            unvisited_request_ids.remove(nearest)
            
            if technician_daily_distance_traveled > truck_max_distance:
                technician_daily_cost += technician_day_cost
                technician_daily_distance_traveled = 0
        
        routes['trucks'].append(truck_routes)
        routes['technicians'].append(technician_routes)
        
        for machine_type in machines:
            if machine_type[0] not in delivery_dates:
                delivery_dates[machine_type[0]] = len(routes['trucks'])  # Assuming each route corresponds to a day
            if delivery_dates[machine_type[0]] is not None and current_location != depot_loc_ID:
                idle_days = max(len(routes['trucks']) - delivery_dates[machine_type[0]] - 1, 0)
                if machine_type[0] not in idle_machine_costs:
                    idle_machine_costs[machine_type[0]] = 0
                idle_machine_costs[machine_type[0]] += idle_days * machine_type[2]
    
    overall_truck_cost += truck_daily_cost
    overall_technician_cost += technician_daily_cost
    
    total_truck_distance = sum(truck_distance_traveled for truck_routes in routes['trucks'] for truck_route in truck_routes for location_id in truck_route)
    total_technician_distance = sum(technician_daily_distance_traveled for technician_routes in routes['technicians'] for technician_route in technician_routes for location_id in technician_route)
    total_cost = calculate_total_cost(overall_truck_cost, overall_technician_cost, idle_machine_costs, routes)
    
    print_output(depot_loc_ID, routes, total_truck_distance, total_technician_distance, total_cost, overall_truck_cost, overall_technician_cost, idle_machine_costs)
    
    return routes, total_truck_distance, total_technician_distance, total_cost



def calculate_total_cost(overall_truck_cost, overall_technician_cost, idle_machine_costs, routes):
    # Calculate total cost using weights provided or default values
    truck_distance_weight = TRUCK_DISTANCE_COST
    truck_day_weight = TRUCK_DAY_COST
    technician_distance_weight = TECHNICIAN_DISTANCE_COST
    technician_day_weight = TECHNICIAN_DAY_COST
    
    # Modify this line to assign weights to each machine ID
    idle_machine_weights = {machine_id: 1 for machine_id in IDLE_MACHINE_COSTS.keys()}  # Example: Assign weight 1 to each machine ID
    
    total_idle_machine_cost = sum(idle_machine_costs.values())  # Sum up idle machine costs
    
    # Calculate the total cost with weighted components
    total_cost = (
        (overall_truck_cost * truck_distance_weight) +
        (overall_technician_cost * technician_distance_weight) +
        (total_idle_machine_cost * sum(idle_machine_weights.values()))  # Weighted sum of idle machine costs
    )
    total_cost += (len(routes['trucks']) * truck_day_weight) + (len(routes['technicians']) * technician_day_weight)
    
    return total_cost


def print_output(depot_loc_ID, routes, total_truck_distance, total_technician_distance, total_cost, overall_truck_cost, overall_technician_cost, idle_machine_costs):
    print("DATASET = VeRoLog solver challenge 2019")
    print("NAME = ", title)
    
    print("TRUCK_DISTANCE =", total_truck_distance)
    print("NUMBER_OF_TRUCK_DAYS =", len(routes['trucks']))
    print("NUMBER_OF_TRUCKS_USED =", max(len(day) for day in routes['trucks']))
    print("TECHNICIAN_DISTANCE =", total_technician_distance)
    print("NUMBER_OF_TECHNICIAN_DAYS =", len(routes['technicians']))
    print("NUMBER_OF_TECHNICIANS_USED =", max(len(day) for day in routes['technicians']))
    
    total_idle_machine_cost = sum(idle_machine_costs.values())  # Sum up idle machine costs
    total_cost = calculate_total_cost(overall_truck_cost, overall_technician_cost, idle_machine_costs, routes)
    
    print("IDLE_MACHINE_COSTS =", total_idle_machine_cost)
    print("TOTAL_COST =", total_cost)
    
    for day, (truck_routes, technician_routes) in enumerate(zip(routes['trucks'], routes['technicians']), start=1):
        print("\nDAY =", day)
        print("NUMBER_OF_TRUCKS =", len(truck_routes))
        for idx, truck_route in enumerate(truck_routes, start=1):
            print(idx, *truck_route)
        print("NUMBER_OF_TECHNICIANS =", len(technician_routes))
        for idx, technician_route in enumerate(technician_routes, start=1):
            print(idx, *technician_route)

# Example usage
solve_vrp(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, TRUCK_DISTANCE_COST, TRUCK_DAY_COST, TRUCK_COST, TECHNICIAN_DISTANCE_COST, TECHNICIAN_DAY_COST, TECHNICIAN_COST, requests, weights)


In [ ]:
output_file_path = f"/Users/juliasmit/Documents/Business Analytics/jaar 3/Combinatorial Optimization/project/output3.txt"  # Construct the output file path
with open(output_file_path, "w") as f:
    f.write(output)  # Write the output string to the file

print("Output saved to:", output_file_path)  # Print the path to the output file

In [ ]:
def solve_vrp(locations, depot, truck_capacity, truck_max_distance, truck_distance_cost, truck_day_cost, truck_cost, technician_distance_cost, technician_day_cost, technician_cost, requests, weights):
    depot_loc_ID = depot[0]  
    unvisited_request_ids = [req[0] for req in requests]  
    current_location = depot_loc_ID  
    routes = {'trucks': [], 'technicians': []}
    
    delivery_dates = {}  
    idle_machine_costs = {}  
    
    day = 0
    
    while unvisited_request_ids:
        day += 1
        truck_routes = []
        technician_routes = []
        
        # Reset variables for the new day
        truck_used = False
        truck_distance_traveled = 0
        truck_daily_distance_traveled = 0
        truck_daily_cost = 0
        overall_truck_cost = 0
        
        technician_home_location = [9999, 9999]  
        
        technician_used = False
        technician_daily_distance_traveled = 0
        technician_daily_cost = 0
        overall_technician_cost = 0

        while unvisited_request_ids and (not truck_used or truck_daily_distance_traveled <= truck_max_distance):
            nearest = nearest_neighbor(current_location, unvisited_request_ids)
            nearest_request = [req for req in requests if req[0] == nearest][0]
            distance_to_nearest = distance_between_locations(current_location, nearest_request[1], locations)
            
            if (not truck_used) or (truck_used and (truck_daily_distance_traveled + distance_to_nearest > truck_max_distance)):
                if truck_used:
                    truck_routes.append([0])  # Return to depot
                truck_used = True
                truck_daily_distance_traveled = 0
                truck_daily_cost = truck_day_cost
                overall_truck_cost += truck_cost
            
            truck_distance_traveled += distance_to_nearest
            truck_daily_distance_traveled += distance_to_nearest
            current_location = nearest_request[1]
            unvisited_request_ids.remove(nearest)
            
            truck_daily_cost += truck_distance_cost * distance_to_nearest
            
            if truck_daily_distance_traveled > truck_max_distance:
                truck_daily_cost += truck_day_cost
                truck_daily_distance_traveled = 0
        
            if truck_routes:
                truck_routes[-1].append(nearest)
            else:
                truck_routes.append([nearest])
        
        while unvisited_request_ids and technician_used:
            nearest = nearest_neighbor(current_location, unvisited_request_ids)
            nearest_request = [req for req in requests if req[0] == nearest][0]
            distance_to_nearest = distance_between_locations(current_location, nearest_request[1], locations)
            
            technician_daily_distance_traveled += distance_to_nearest
            technician_daily_cost += technician_distance_cost * distance_to_nearest
            
            technician_routes[-1].append(nearest)
            current_location = nearest_request[1]
            unvisited_request_ids.remove(nearest)
            
            if technician_daily_distance_traveled > truck_max_distance:
                technician_daily_cost += technician_day_cost
                technician_daily_distance_traveled = 0
        
        routes['trucks'].append(truck_routes)
        routes['technicians'].append(technician_routes)
        
        for machine_type in machines:
            if machine_type[0] not in delivery_dates:
                delivery_dates[machine_type[0]] = len(routes['trucks'])  # Assuming each route corresponds to a day
            if delivery_dates[machine_type[0]] is not None and current_location != depot_loc_ID:
                idle_days = max(len(routes['trucks']) - delivery_dates[machine_type[0]] - 1, 0)
                if machine_type[0] not in idle_machine_costs:
                    idle_machine_costs[machine_type[0]] = 0
                idle_machine_costs[machine_type[0]] += idle_days * machine_type[2]
    
    overall_truck_cost += truck_daily_cost
    overall_technician_cost += technician_daily_cost
    
    total_truck_distance = sum(truck_distance_traveled for truck_routes in routes['trucks'] for truck_route in truck_routes for location_id in truck_route)
    total_technician_distance = sum(technician_daily_distance_traveled for technician_routes in routes['technicians'] for technician_route in technician_routes for location_id in technician_route)
    total_cost = calculate_total_cost(overall_truck_cost, overall_technician_cost, idle_machine_costs, routes, weights)
    
    return print_output(depot_loc_ID, routes, total_truck_distance, total_technician_distance, total_cost, overall_truck_cost, overall_technician_cost, weights, idle_machine_costs)

def calculate_total_cost(overall_truck_cost, overall_technician_cost, idle_machine_costs, routes, weights):
    # Calculate total cost using weights provided or default values
    truck_distance_weight = weights.get('truck_distance', 1)
    truck_day_weight = weights.get('truck_day', 100)
    technician_distance_weight = weights.get('technician_distance', 1)
    technician_day_weight = weights.get('technician_day', 100)
    idle_machine_weight = weights.get('idle_machine', 100000)
    
    total_idle_machine_cost = sum(idle_machine_costs.values())  # Sum up idle machine costs
    total_cost = (overall_truck_cost * truck_distance_weight) + (overall_technician_cost * technician_distance_weight) + (total_idle_machine_cost * idle_machine_weight)
    total_cost += (len(routes['trucks']) * truck_day_weight) + (len(routes['technicians']) * technician_day_weight)
    
    return total_cost

def print_output(depot_loc_ID, routes, total_truck_distance, total_technician_distance, total_cost, overall_truck_cost, overall_technician_cost, weights, idle_machine_costs):
    output = ""
    output += "DATASET = VeRoLog solver challenge 2019\n"
    output += "NAME = Instance 1\n\n"
    
    output += f"TRUCK_DISTANCE = {total_truck_distance}\n"
    output += f"NUMBER_OF_TRUCK_DAYS = {len(routes['trucks'])}\n"
    output += f"NUMBER_OF_TRUCKS_USED = {max(len(day) for day in routes['trucks'])}\n"
    output += f"TECHNICIAN_DISTANCE = {total_technician_distance}\n"
    output += f"NUMBER_OF_TECHNICIAN_DAYS = {len(routes['technicians'])}\n"
    output += f"NUMBER_OF_TECHNICIANS_USED = {max(len(day) for day in routes['technicians'])}\n\n"
    
    total_idle_machine_cost = sum(idle_machine_costs.values())  # Sum up idle machine costs
    total_cost = calculate_total_cost(overall_truck_cost, overall_technician_cost, idle_machine_costs, routes, weights)
    
    output += f"IDLE_MACHINE_COSTS = {total_idle_machine_cost}\n"
    output += f"TOTAL_COST = {total_cost}\n\n"
    
    for day, (truck_routes, technician_routes) in enumerate(zip(routes['trucks'], routes['technicians']), start=1):
        output += f"\nDAY = {day}\n"
        output += f"NUMBER_OF_TRUCKS = {len(truck_routes)}\n"
        for idx, truck_route in enumerate(truck_routes, start=1):
            output += f"{idx} {' '.join(map(str, truck_route))}\n"
        output += f"NUMBER_OF_TECHNICIANS = {len(technician_routes)}\n"
        for idx, technician_route in enumerate(technician_routes, start=1):
            output += f"{idx} {' '.join(map(str, technician_route))}\n"
    
    return output

# Example usage
weights = {'truck_distance': 1, 'truck_day': 100, 'technician_distance': 1, 'technician_day': 100, 'idle_machine': 100000}
output = solve_vrp(locations, depot, TRUCK_CAPACITY, TRUCK_MAX_DISTANCE, TRUCK_DISTANCE_COST, TRUCK_DAY_COST, TRUCK_COST, TECHNICIAN_DISTANCE_COST, TECHNICIAN_DAY_COST, TECHNICIAN_COST, requests, weights)

c
